In [ ]:
import json
import numpy as np
import string
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets sentencepiece

!pip uninstall -q -y transformers accelerate
!pip install -q transformers accelerate

!pip install -q evaluate

!pip install -q git+https://github.com/huggingface/accelerate

!git clone https://github.com/huggingface/transformers.git
!cd transformers
!pip install -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Cloning into 'transformers'...
remote: Enumerating objects: 195162, done.
remote: Counting objects: 100% (18271/18271), done.
remote: Compressing objects: 100% (1357/1357), done.
remote: Total 195162 (delta 17926), reused 16941 (delta 16907), pack-reused 176891
Receiving objects: 100% (195162/1

In [ ]:
# from datasets import load_dataset

# data_files = {"train": "/content/drive/MyDrive/train_v9.json", "validation": "/content/drive/MyDrive/dev_v9.json"}
# dataset = load_dataset("json", data_files=data_files , encoding = 'UTF-8-sig')

In [ ]:
with open('/content/drive/MyDrive/extra_translations.json', 'r', encoding = 'UTF-8-sig') as f:
  qas = json.load(f)

new_data_list = []
for qa in qas:
  new_data = {}
  new_data['question'] = qa['question']
  new_data['pq_id'] = qa['pq_id']
  new_data['context'] = qa['context']
  new_data['answers'] = qa['answers']

  new_data_list.append(new_data)

with open('/content/drive/MyDrive/extra_translations2.json', 'w', encoding = 'utf-8-sig') as f:
  json.dump(new_data_list, f, ensure_ascii= False)

In [ ]:
from datasets import load_dataset

data_files = {"train": "/content/drive/MyDrive/extra_translations2.json", "validation": "/content/drive/MyDrive/dev_v9.json"}
dataset = load_dataset("json", data_files=data_files , encoding = 'UTF-8-sig')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['pq_id', 'answers', 'question', 'context'],
        num_rows: 26805
    })
    validation: Dataset({
        features: ['pq_id', 'answers', 'question', 'context'],
        num_rows: 1528
    })
})

In [ ]:
from transformers import AutoTokenizer

# model_checkpoint = "m3hrdadfi/albert-fa-base-v2"
# model_checkpoint = "/content/drive/MyDrive/PQuAD_FineTuned/roberta_pquad/checkpoint-8000"
model_checkpoint = "/content/drive/MyDrive/roberta_extra_translation"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]

        start_char = answer[0]["start_char"]
        end_char = answer[0]["start_char"] + len(answer[0]['text'])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = dataset["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=dataset["train"].column_names,
)
len(dataset["train"]), len(train_dataset)

Map:   0%|          | 0/26805 [00:00<?, ? examples/s]

(26805, 26882)

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["pq_id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = dataset["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)
len(dataset["validation"]), len(validation_dataset)

Map:   0%|          | 0/1528 [00:00<?, ? examples/s]

(1528, 1528)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
from tqdm.auto import tqdm
import collections

n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["pq_id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = []
    for ex in examples:
      new_ex = {}
      new_ex['id'] = ex["pq_id"]
      new_ex['answers'] = {}
      new_ex['answers']['answer_start'] = [ex['answers'][0]['start_char']]
      new_ex['answers']['text'] = [ex['answers'][0]['text']]
      theoretical_answers.append(new_ex)


    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    '/content/drive/MyDrive/roberta_extra_translation',
    evaluation_strategy="no",
    save_strategy="no",
    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True,
)

# args = TrainingArguments(
#     '/content/drive/MyDrive/parsbert_mydata',
#     evaluation_strategy = "epoch",
#     learning_rate=lr,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=1,

#     weight_decay=0.01)

In [ ]:
from transformers import AutoModelForQuestionAnswering

# model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint, use_safetensors=True)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,0.074400
1000,0.066000
1500,0.057600
2000,0.085400
2500,0.106900
3000,0.166000
3500,0.258900
4000,0.196500
4500,0.176900
5000,0.175600


TrainOutput(global_step=6722, training_loss=0.14085877987998682, metrics={'train_runtime': 2066.5859, 'train_samples_per_second': 26.016, 'train_steps_per_second': 3.253, 'total_flos': 1.0536269121865728e+16, 'train_loss': 0.14085877987998682, 'epoch': 2.0})

In [ ]:
trainer.save_model()

In [ ]:
import evaluate

metric = evaluate.load("squad")
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, dataset["validation"])

  0%|          | 0/1528 [00:00<?, ?it/s]

{'exact_match': 68.32460732984293, 'f1': 74.14657649830038}